In [1]:
import numpy as np
import pandas as pd
import os
import ipython_bell
import random

from astropy.io import fits
from tqdm.notebook import tqdm

In [2]:
complete_data = pd.read_csv('Data/all_data_sim.csv')['COADD_OBJECT_ID'] #only column with ID
print(len(complete_data))
data = complete_data.sample(n = 1000) #select random sample

20265


In [3]:
y = np.zeros((1,3,46,46))
cutout_path = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/lens_cutouts/'
bands = ['g', 'r', 'i']
for point in tqdm(data):
    for band in bands:
        complete_path = cutout_path + str(point) + '_' + band + '.fits'
        if(os.path.exists(complete_path) == False):
            continue
        with fits.open(complete_path) as hdul:
            if complete_path[-6:] == 'g.fits': cutout_g = hdul[0].data
            if complete_path[-6:] == 'i.fits': cutout_i = hdul[0].data
            if complete_path[-6:] == 'r.fits': cutout_r = hdul[0].data
    ele_cut = np.array([cutout_g, cutout_r, cutout_i])
    ele_cut = np.rot90(ele_cut, k=random.randint(0, 4), axes=(1, 2))
    if(random.randint(0, 1) == 1):
        ele_cut = np.flip(ele_cut, 2)
    y = np.append(y, [ele_cut], axis = 0)
y = np.delete(y, 0, axis = 0)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [4]:
primary = fits.PrimaryHDU()
image = fits.ImageHDU(y, name="IMAGE")
hdu_list = fits.HDUList([primary, image])

In [5]:
bell -n notify hdu_list.writeto('Data/Sim_complete/negative_cases.fits', overwrite=True)

In [6]:
bell -n say